In [14]:
import obspy
for i in range(0,10):
    obspy.read('/home/earthquakes1/homes/Rebecca/phd/data/2005_2018_global_m5/20050131_173151.a/data/BFZ/NZ.BFZ.10.HHZ__20050131T172651Z__20050131T173651Z.mseed')